Ok - new tree with all isolates

In [4]:
import glob
import re
import pandas as pd

Ok - trying to use prokka - roary to make a pangenome

Made a conda environment conda create --name cpowgs
Then conda install -c conda-forge -c bioconda -c defaults prokka
then
conda config --add channels r\
conda config --add channels defaults\
conda config --add channels conda-forge\
conda config --add channels bioconda\
conda install roary\
then\
conda install jupyterlab (I know, wrong order)\
then\
conda install raxml\
then\
conda install snp-sites\
then\
conda install pandas\
then\
conda install rpy2\
then\
conda install r-tidyverse\

In [5]:
workdir = '/atium/Data/nanopore/cpowgs/trees/200811_timp_tree/'
%cd /atium/Data/Nanopore/cpowgs/trees/200811_timp_tree

/atium/Data/Nanopore/cpowgs/trees/200811_timp_tree


Ok - I need to pull down the genbank files for Enterobacter cloacae: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
E. Coli: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
and K. Pneumo: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz

In [6]:
klpn=pd.read_excel("from_trish_isolates.xlsx", sheet_name="KLPN", skiprows=3, header=None, usecols="A:F", names=["id", "cp", "cp_gene", "mlst", "repA", "year"])
encl=pd.read_excel("from_trish_isolates.xlsx", sheet_name="ENCL", skiprows=2, header=None, usecols="A:E", names=["id", "cp", "cp_gene", "repA", "year"])
esco=pd.read_excel("from_trish_isolates.xlsx", sheet_name="ESCO", skiprows=2, header=None, usecols="A:F", names=["id", "cp", "cp_gene", "mlst", "repA", "year"])

In [10]:
assemblydir='/atium/Data/Nanopore/cpowgs/assemblies/'
illdirall='/atium/Data/Nanopore/cpowgs/illumina_*/'
fullill=glob.glob(illdirall+"*.fastq.gz")

In [8]:
def assembly_find(meta, label):
    """This function finds the pilon or illumina data for the samples from a meta file"""
    meta["pilon"]=""
    meta["spades"]=""
    meta["illR1"]=""
    meta["illR2"]=""
    print(label+":")
    for i in meta.index:
        passemble=glob.glob(assemblydir+"pilon/"+label.upper()+"_"+str(meta.id[i])+".pilon*")
        if passemble:
            meta.loc[i, 'pilon']=passemble
        else:
            #look for spades
            sassemble=glob.glob(label+"_spades/"+str(meta.id[i])+"/scaffolds.fasta")
            if sassemble:
                meta.loc[i, 'spades']=sassemble
            else:
                searchstring_r1=str(meta.id[i])+"[^0-9.].*R1"
                searchstring_r2=str(meta.id[i])+"[^0-9.].*R2"
                ill_r1=[f for f in fullill if re.search(searchstring_r1, f)]
                ill_r2=[f for f in fullill if re.search(searchstring_r2, f)]
                if ill_r1:
                    #print(ill)
                    meta.loc[i, 'illR1']=ill_r1[0]
                    meta.loc[i, 'illR2']=ill_r2[0]
                    #print("Yes Illumina in dir1")
                else:
                    print("Missing data: "+str(meta.id[i]))
    return(meta)

%mkdir refs
%cd refs
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
!gunzip *gz
%mv *gbff ecoli_ref.gb
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
!gunzip *gz
%mv *gbff encl_ref.gb
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz
!gunzip *gz
%mv *gbff kpn_ref.gb
%cd ..

In [11]:
klpn=assembly_find(klpn, "klpn")
esco=assembly_find(esco, "esco")
encl=assembly_find(encl, "encl")


klpn:
esco:
encl:


In [14]:
encl[encl.pilon=='']

,id,cp,cp_gene,repA,year,pilon,spades,illR1,illR2
0,1,Non-CP,NaN,NaN,2016,,,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...
1,2,Non-CP,NaN,NaN,2016,,encl_spades/2/scaffolds.fasta,,
2,3,Non-CP,NaN,NaN,2016,,,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...
3,12,Non-CP,NaN,NaN,2016,,encl_spades/12/scaffolds.fasta,,
4,14,Non-CP,NaN,NaN,2016,,,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...,/atium/Data/Nanopore/cpowgs/illumina_marcc/KLP...
...,...,...,...,...,...,...,...,...,...
77,603,Non-CP,NaN,NaN,2018,,encl_spades/603/scaffolds.fasta,,
78,614,CP,NaN,NaN,2018,,encl_spades/614/scaffolds.fasta,,
79,624,Non-CP,NaN,NaN,2018,,encl_spades/624/scaffolds.fasta,,
80,628,Non-CP,NaN,NaN,2018,,encl_spades/628/scaffolds.fasta,,
